### Agentic RAG
 : 질문에 따라 문서를 검색하여 답변(RAG)하거나, 인터넷 검색 도구(Tavily Search:)를 활용하여 답변하는 에이전트

**참고**

- **Agentic RAG**  : RAG 를 수행하되, Agent 를 활용하여 RAG 를 수행하는 에이전트

LangChain에서는 에이전트가 여러 툴을 조합하여 복잡한 문제를 해결하는 능력을 가지고 있기 때문에, Tool은 에이전트가 활용할 수 있는 기능적 요소
Agent 가 활용할 도구를 정의하여 Agent가 추론(reasoning)을 수행할 때 활용하도록 만들 수 있음

-  **도구(Tools)** :특정 작업(계산, 검색, API 호출 등)을 수행하는 도구
- **에이전트** : 사용자의 요청을 분석하고 적절한 툴을 사용하여 문제를 해결하는 주체


### 웹 검색도구: SerpAPI
- [SerpAPI API 발급받기](https://serpapi.com/)

`.env`에 환경변수 등록

- `SERPAPI_API_KEY=발급 받은 SerpAPI API KEY 입력`

In [ ]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu pypdf google-search-results

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv("/content/.env", override=True)

`search.run` 함수는 주어진 문자열에 대한 검색을 실행

`run()` 함수에 검색하고 싶은 검색어를 넣어 검색을 수행


In [ ]:
from langchain_community.utilities import SerpAPIWrapper
# 검색 결과
search = SerpAPIWrapper()
search.run("오늘 인천 날씨를 알려주세요")

In [26]:
from langchain_community.utilities import SerpAPIWrapper

def search_web():
    search = SerpAPIWrapper()

    def run_with_source(query: str) -> str:
        results = search.results(query)
        organic = results.get("organic_results", [])
        formatted = []
        for r in organic[:5]:
            title = r.get("title")
            link = r.get("link")
            source = r.get("source")
            snippet = r.get("snippet")  # ✅ snippet 추가
            if link:
                formatted.append(f"- [{title}]({link}) ({source})\n  {snippet}")
            else:
                formatted.append(f"- {title} (출처: {source})\n  {snippet}")
        return "\n".join(formatted) if formatted else "검색 결과가 없습니다."

    return Tool(
        name="web_search",
        func=run_with_source,
        description="실시간 뉴스 및 웹 정보를 검색할 때 사용합니다. 결과는 제목+출처+링크+간단요약(snippet) 형태로 반환됩니다."
    )

### 문서 기반 문서 검색 도구: Retriever

내가 넣은 문서에 대해 조회를 수행할 retriever도 생성

**실습에 활용한 문서**

초보 투자자를 위한 증권과 투자 따라잡기.pdf

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("/content/초보 투자자를 위한 증권과 투자 따라잡기.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()

 `retriever` 객체의 `invoke()` 를 사용하여 사용자의 질문에 대한 가장 **관련성 높은 문서** 를 찾는 데 사용


In [ ]:
# 문서에서 관련성 높은 문서를 가져옴
retriever.invoke("채권에 대한 개념을 알려줘.")

이제 우리가 검색을 수행할 인덱스를 채웠으므로, 이를 에이전트가 제대로 사용할 수 있는 도구로 쉽게 변환 가능


`create_retriever_tool` 함수로 `retriever` 를 도구로 변환

In [29]:
from langchain.tools.retriever import create_retriever_tool


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야함!!
)

### Agent 가 사용할 도구 목록 정의

Agent 가 사용할 도구 목록:  
  
`tools` 리스트는 `search`와 `retriever_tool`을 포함

In [30]:
from langchain.agents import Tool
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search_web(), retriever_tool]

#### Agent 생성

Agent 가 활용할 LLM을 정의하고, Agent 가 참고할 Prompt 를 정의

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

Tool Calling Agent 를 생성

In [32]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

마지막으로, 생성한 `agent` 를 실행하는 `AgentExecutor` 를 생성
- (참고)`verbose=False` 로 설정하여 중간 단계 출력을 생략

In [33]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

#### 에이전트 실행하기

현재 이러한 모든 질의는 **상태(Stateless) 가 없는** 질의 == 이전 상호작용을 기억하지 못함


`agent_executor` 객체의 `invoke` 메소드는 딕셔너리 형태의 인자를 받아 처리

In [34]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()

In [ ]:
result = agent_executor.stream(
    {"input": "2025년 넷플릭스 신작 알려주세요"}
)

for step in result:
    # 1. Agent가 선택한 도구(action) 출력
    if "actions" in step:
        for action in step["actions"]:
            print(f"\n[도구 호출] {action.tool}")
            tool_input = action.tool_input
            if isinstance(tool_input, dict):
                for k, v in tool_input.items():
                    print(f"  {k}: {v}")
            else:
                print(f"  입력값: {tool_input}")

    # 2. 도구 실행 결과(observation) 출력
    if "observations" in step:
        for obs in step["observations"]:
            print(f"\n[도구 응답] {obs}")

    # 3. 최종 출력 결과
    if "output" in step:
        print("\n[최종 답변]")
        print(step["output"])


`agent_executor` 객체의 `invoke` 메소드를 사용하여, 질문을 입력으로 제공


In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "선물과 주식에 대한 차이점을 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

## 이전 대화내용 기억하는 Agent

`RunnableWithMessageHistory`: 이전의 대화내용을 기억하기 위해 사용  

`AgentExecutor`를 `RunnableWithMessageHistory`로 감싸줌

In [37]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "선물과 주식에 대한 차이점을 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)

## Agent 템플릿

In [ ]:
# PyMuPDF : 텍스트, 이미지, 주석, 레이아웃 등 PDF의 모든 요소 처리 가능한 패키지
! pip install PyMuPDF

In [43]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의 ##########

### 1-1. Search 도구 ###
def search_web():
    search = SerpAPIWrapper()

    def run_with_source(query: str) -> str:
        results = search.results(query)
        organic = results.get("organic_results", [])
        formatted = []
        for r in organic[:5]:
            title = r.get("title")
            link = r.get("link")
            source = r.get("source")
            snippet = r.get("snippet")  # ✅ snippet 추가
            if link:
                formatted.append(f"- [{title}]({link}) ({source})\n  {snippet}")
            else:
                formatted.append(f"- {title} (출처: {source})\n  {snippet}")
        return "\n".join(formatted) if formatted else "검색 결과가 없습니다."

    return Tool(
        name="web_search",
        func=run_with_source,
        description="실시간 뉴스 및 웹 정보를 검색할 때 사용합니다. 결과는 제목+출처+링크+간단요약(snippet) 형태로 반환됩니다."
    )

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("/content/초보 투자자를 위한 증권과 투자 따라잡기.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search_web(), retriever_tool]

########## 2. LLM 을 정의  ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input":  "국내 증권 시장의 역사를 문서에서 찾아서 추천해줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "넷플릭스 TV 프로그램 흑백요리사 최종 우승자를 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    # 1. Agent가 선택한 도구(action) 출력
    if "actions" in step:
        for action in step["actions"]:
            print(f"\n[도구 호출] {action.tool}")
            tool_input = action.tool_input
            if isinstance(tool_input, dict):
                for k, v in tool_input.items():
                    print(f"  {k}: {v}")
            else:
                print(f"  입력값: {tool_input}")

    # 2. 도구 실행 결과(observation) 출력
    if "observations" in step:
        for obs in step["observations"]:
            print(f"\n[도구 응답] {obs}")

    # 3. 최종 출력 결과
    if "output" in step:
        print("\n[최종 답변]")
        print(step["output"])

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 우승자의 소감도 찾아줘."},
    # 세션 ID를 설정
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않음
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    # 1. Agent가 선택한 도구(action) 출력
    if "actions" in step:
        for action in step["actions"]:
            print(f"\n[도구 호출] {action.tool}")
            tool_input = action.tool_input
            if isinstance(tool_input, dict):
                for k, v in tool_input.items():
                    print(f"  {k}: {v}")
            else:
                print(f"  입력값: {tool_input}")

    # 2. 도구 실행 결과(observation) 출력
    if "observations" in step:
        for obs in step["observations"]:
            print(f"\n[도구 응답] {obs}")

    # 3. 최종 출력 결과
    if "output" in step:
        print("\n[최종 답변]")
        print(step["output"])

#### [실습] PDF를 활용한 RAG + 실시간 웹 검색을 활용한 챗봇 구축
 - 자유주제 가능
  
 - (예시) 요리 레시피 검색 및 추천 에이전트  
    목표: PDF로 제공된 요리책에서 원하는 레시피를 검색하고, 찾을 수 없으면 웹에서 정보를 검색해 추천 요리법을 제공하는 시스템을 구축
    
 - (예시) 여행 정보 검색 및 추천 에이전트  
   목표: PDF 문서로 제공된 여행 가이드북에서 특정 목적지에 대한 정보를 검색하고, 부족한 정보는 웹에서 검색해 종합적인 여행 계획을 제시하는 시스템을 구축

 - (예시) 논문 검색 및 관련 연구 서베이 에이전트
   목표: 공모전 혹은 프로젝트에 사용할 딥러닝 모델을 논문 RAG 문서로 활용하여, 논문 질의 응답을 받은 후 부족한 정보다 비교 모델을 웹에서 검색해 사용할 모델들을 정리해주는 시스템을 구축

In [ ]:
# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의 ##########
### 1-1. Search 도구 ###
def search_web():
    search = SerpAPIWrapper()

    def run_with_source(query: str) -> str:
        results = search.results(query)
        organic = results.get("organic_results", [])
        formatted = []
        for r in organic[:5]:
            title = r.get("title")
            link = r.get("link")
            source = r.get("source")
            snippet = r.get("snippet")  # ✅ snippet 추가
            if link:
                formatted.append(f"- [{title}]({link}) ({source})\n  {snippet}")
            else:
                formatted.append(f"- {title} (출처: {source})\n  {snippet}")
        return "\n".join(formatted) if formatted else "검색 결과가 없습니다."

    return Tool(
        name="web_search",
        func=run_with_source,
        description="실시간 뉴스 및 웹 정보를 검색할 때 사용합니다. 결과는 제목+출처+링크+간단요약(snippet) 형태로 반환됩니다."
    )

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("./----.pdf")

# 텍스트 분할기를 사용하여 문서를 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성
retriever = vector.as_retriever()


retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가 ###
# tools 리스트에 search와 retriever_tool을 추가
tools = [search_web(), retriever_tool]

########## 2. LLM 을 정의  ##########
# LLM 모델을 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의##########

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의 ##########

# 에이전트를 생성
# llm, tools, prompt를 인자로 사용
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가 ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
agent_stream_parser = AgentStreamParser()